<a href="https://colab.research.google.com/github/carlos24520/contabilidad/blob/main/contabilidad%20pathon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
# Install Streamlit if not already installed
!pip install streamlit

import streamlit as st
import pandas as pd
from datetime import datetime

# Configuración de la página
st.set_page_config(page_title="Sistema Contable Pro Elite", layout="wide")

# Inicialización de estados (Base de datos temporal)
if 'movimientos' not in st.session_state:
    st.session_state.movimientos = []
if 'cierres' not in st.session_state:
    st.session_state.cierres = []

# --- LÓGICA DE NEGOCIO ---
def calcular_iva(monto, tipo):
    iva = round(monto * 0.19)
    if tipo == "Compra":
        return "IVA Crédito Fiscal", iva, 0, "Activo"
    else:
        return "IVA Débito Fiscal", 0, iva, "Pasivo"

# --- INTERFAZ DE USUARIO ---
st.title("📊 Contabilidad Pro Elite Python v2026")
st.subheader("Balance de 8 Columnas y Módulo SII")

# Panel de entrada
with st.expander("➕ Registrar Nuevo Asiento", expanded=True):
    col1, col2, col3, col4 = st.columns([1, 2, 2, 1])

    with col1:
        fecha = st.date_input("Fecha", datetime.now())
    with col2:
        cuenta = st.text_input("Cuenta", placeholder="Ej: Mercaderías")
    with col3:
        glosa = st.text_input("Glosa", placeholder="Descripción")
    with col4:
        clase = st.selectbox("Clasificación", ["Activo", "Pasivo", "Gasto", "Ingreso"])

    c_debe, c_haber, c_iva = st.columns(3)
    with c_debe:
        debe = st.number_input("Debe $", min_value=0, step=100)
    with c_haber:
        haber = st.number_input("Haber $", min_value=0, step=100)
    with c_iva:
        st.write("¿Calcular IVA?")
        if st.button("Aplicar 19% IVA"):
            # Lógica rápida para agregar el IVA basado en el último ingreso
            monto = debe if debe > 0 else haber
            tipo = "Compra" if debe > 0 else "Venta"
            nom_iva, d_iva, h_iva, cl_iva = calcular_iva(monto, tipo)
            st.session_state.movimientos.append(
                {"Fecha": fecha, "Cuenta": nom_iva, "Glosa": f"IVA de {glosa}",
                 "Debe": d_iva, "Haber": h_iva, "Clasificación": cl_iva}
            )
            st.toast("IVA agregado correctamente")

    if st.button("💾 Guardar Asiento", use_container_width=True):
        if cuenta:
            st.session_state.movimientos.append({
                "Fecha": fecha, "Cuenta": cuenta, "Glosa": glosa,
                "Debe": debe, "Haber": haber, "Clasificación": clase
            })
            st.success("Asiento registrado")
        else:
            st.error("Falta el nombre de la cuenta")

# --- PROCESAMIENTO DE DATOS ---
if st.session_state.movimientos:
    df = pd.DataFrame(st.session_state.movimientos)

    # 1. LIBRO DIARIO
    st.divider()
    st.subheader("1. Libro Diario")
    st.dataframe(df, use_container_width=True)

    # 2. LIBRO MAYOR (ESQUEMAS EN T)
    st.subheader("2. Libro Mayor")
    cuentas_unicas = df['Cuenta'].unique()
    cols_mayor = st.columns(len(cuentas_unicas) if len(cuentas_unicas) < 4 else 4)

    for i, c in enumerate(cuentas_unicas):
        with cols_mayor[i % 4]:
            st.info(f"**{c}**")
            df_c = df[df['Cuenta'] == c]
            st.table(df_c[['Debe', 'Haber']])
            st.write(f"Sumas: **{df_c['Debe'].sum()} | {df_c['Haber'].sum()}**")

    # 3. BALANCE DE 8 COLUMNAS
    st.subheader("3. Balance de 8 Columnas")

    balance = df.groupby(['Cuenta', 'Clasificación']).agg({'Debe':'sum', 'Haber':'sum'}).reset_index()

    # Saldos
    balance['Deudor'] = (balance['Debe'] - balance['Haber']).apply(lambda x: x if x > 0 else 0)
    balance['Acreedor'] = (balance['Haber'] - balance['Debe']).apply(lambda x: x if x > 0 else 0)

    # Inventario y Resultado
    balance['Activo'] = balance.apply(lambda x: x['Deudor'] if x['Clasificación'] == 'Activo' else 0, axis=1)
    balance['Pasivo'] = balance.apply(lambda x: x['Acreedor'] if x['Clasificación'] == 'Pasivo' else 0, axis=1)
    balance['Pérdida'] = balance.apply(lambda x: x['Deudor'] if x['Clasificación'] == 'Gasto' else 0, axis=1)
    balance['Ganancia'] = balance.apply(lambda x: x['Acreedor'] if x['Clasificación'] == 'Ingreso' else 0, axis=1)

    st.table(balance)

    # Totales y Utilidad
    t = balance.sum(numeric_only=True)
    utilidad_inv = t['Activo'] - t['Pasivo']
    utilidad_res = t['Ganancia'] - t['Pérdida']

    c1, c2 = st.columns(2)
    with c1:
        st.metric("Resultado por Inventario", f"${utilidad_inv:,.0f}")
    with c2:
        st.metric("Resultado por Ganancia/Pérdida", f"${utilidad_res:,.0f}")

    if utilidad_inv == utilidad_res:
        st.success("✅ ¡Balance Cuadrado!")
    else:
        st.warning("⚠️ El balance no cuadra. Revisa las clasificaciones.")

    # --- MÓDULO SII ---
    st.divider()
    col_f29, col_f22 = st.columns(2)

    with col_f29:
        st.subheader("📑 Simulador F29")
        iva_d = df[df['Cuenta'].str.contains("Débito", case=False)]['Haber'].sum()
        iva_c = df[df['Cuenta'].str.contains("Crédito", case=False)]['Debe'].sum()
        ingresos = t['Ganancia']
        ppm = round(ingresos * 0.01)
        total_pago = max(0, iva_d - iva_c) + ppm
        st.write(f"IVA a Pagar: ${max(0, iva_d - iva_c)}")
        st.write(f"PPM (1%): ${ppm}")
        st.markdown(f"**Total Tesorería: ${total_pago}**")

else:
    st.info("Aún no hay datos. Registra un asiento arriba.")

# Sidebar - Herramientas
with st.sidebar:
    st.header("Controles")
    if st.button("🗑️ Resetear Sistema"):
        st.session_state.movimientos = []
        st.rerun()

    if st.session_state.movimientos:
        csv = pd.DataFrame(st.session_state.movimientos).to_csv(index=False).encode('utf-8')
        st.download_button("📥 Descargar CSV", csv, "contabilidad.csv", "text/csv")

2026-02-24 17:38:08.460 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-24 17:38:08.461 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-24 17:38:08.462 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-24 17:38:08.466 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-24 17:38:08.468 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-24 17:38:08.471 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-24 17:38:08.474 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-24 17:38:08.475 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [20]:
!pip install pyngrok
from pyngrok import ngrok

# 1. PEGA AQUÍ TU TOKEN (Solo se hace una vez)
# ¡IMPORTANTE!: Reemplaza 'TU_AUTHTOKEN_AQUI_ENTRE_COMILLAS' con tu token real de ngrok.
ngrok.set_auth_token("3A5ylB0op0k2iL68de7ztNgBVh5_5Fb2KhvBNQQd5RR3yczLs")

# 2. Ahora sí, levanta el túnel
public_url = ngrok.connect(8501).public_url
print("Tu sistema contable está vivo en:", public_url)

Tu sistema contable está vivo en: https://jonnie-dilapidated-aylin.ngrok-free.dev


In [22]:
%%writefile app.py

import streamlit as st
import pandas as pd
from datetime import datetime

# Configuración de la página
st.set_page_config(page_title="Sistema Contable Pro Elite", layout="wide")

# Inicialización de estados (Base de datos temporal)
if 'movimientos' not in st.session_state:
    st.session_state.movimientos = []
if 'cierres' not in st.session_state:
    st.session_state.cierres = []

# --- LÓGICA DE NEGOCIO ---
def calcular_iva(monto, tipo):
    iva = round(monto * 0.19)
    if tipo == "Compra":
        return "IVA Crédito Fiscal", iva, 0, "Activo"
    else:
        return "IVA Débito Fiscal", 0, iva, "Pasivo"

# --- INTERFAZ DE USUARIO ---
st.title("📊 Contabilidad Pro Elite Python v2026")
st.subheader("Balance de 8 Columnas y Módulo SII")

# Panel de entrada
with st.expander("➕ Registrar Nuevo Asiento", expanded=True):
    col1, col2, col3, col4 = st.columns([1, 2, 2, 1])

    with col1:
        fecha = st.date_input("Fecha", datetime.now())
    with col2:
        cuenta = st.text_input("Cuenta", placeholder="Ej: Mercaderías")
    with col3:
        glosa = st.text_input("Glosa", placeholder="Descripción")
    with col4:
        clase = st.selectbox("Clasificación", ["Activo", "Pasivo", "Gasto", "Ingreso"])

    c_debe, c_haber, c_iva = st.columns(3)
    with c_debe:
        debe = st.number_input("Debe $", min_value=0, step=100)
    with c_haber:
        haber = st.number_input("Haber $", min_value=0, step=100)
    with c_iva:
        st.write("¿Calcular IVA?")
        if st.button("Aplicar 19% IVA"):
            # Lógica rápida para agregar el IVA basado en el último ingreso
            monto = debe if debe > 0 else haber
            tipo = "Compra" if debe > 0 else "Venta"
            nom_iva, d_iva, h_iva, cl_iva = calcular_iva(monto, tipo)
            st.session_state.movimientos.append(
                {"Fecha": fecha, "Cuenta": nom_iva, "Glosa": f"IVA de {glosa}",
                 "Debe": d_iva, "Haber": h_iva, "Clasificación": cl_iva}
            )
            st.toast("IVA agregado correctamente")

    if st.button("💾 Guardar Asiento", use_container_width=True):
        if cuenta:
            st.session_state.movimientos.append({
                "Fecha": fecha, "Cuenta": cuenta, "Glosa": glosa,
                "Debe": debe,
                "Haber": haber, "Clasificación": clase
            })
            st.success("Asiento registrado")
        else:
            st.error("Falta el nombre de la cuenta")

# --- PROCESAMIENTO DE DATOS ---
if st.session_state.movimientos:
    df = pd.DataFrame(st.session_state.movimientos)

    # 1. LIBRO DIARIO
    st.divider()
    st.subheader("1. Libro Diario")
    st.dataframe(df, use_container_width=True)

    # 2. LIBRO MAYOR (ESQUEMAS EN T)
    st.subheader("2. Libro Mayor")
    cuentas_unicas = df['Cuenta'].unique()
    cols_mayor = st.columns(len(cuentas_unicas) if len(cuentas_unicas) < 4 else 4)

    for i, c in enumerate(cuentas_unicas):
        with cols_mayor[i % 4]:
            st.info(f"**{c}**")
            df_c = df[df['Cuenta'] == c]
            st.table(df_c[['Debe', 'Haber']])
            st.write(f"Sumas: **{df_c['Debe'].sum()} | {df_c['Haber'].sum()}**")

    # 3. BALANCE DE 8 COLUMNAS
    st.subheader("3. Balance de 8 Columnas")

    balance = df.groupby(['Cuenta', 'Clasificación']).agg({'Debe':'sum', 'Haber':'sum'}).reset_index()

    # Saldos
    balance['Deudor'] = (balance['Debe'] - balance['Haber']).apply(lambda x: x if x > 0 else 0)
    balance['Acreedor'] = (balance['Haber'] - balance['Debe']).apply(lambda x: x if x > 0 else 0)

    # Inventario y Resultado
    balance['Activo'] = balance.apply(lambda x: x['Deudor'] if x['Clasificación'] == 'Activo' else 0, axis=1)
    balance['Pasivo'] = balance.apply(lambda x: x['Acreedor'] if x['Clasificación'] == 'Pasivo' else 0, axis=1)
    balance['Pérdida'] = balance.apply(lambda x: x['Deudor'] if x['Clasificación'] == 'Gasto' else 0, axis=1)
    balance['Ganancia'] = balance.apply(lambda x: x['Acreedor'] if x['Clasificación'] == 'Ingreso' else 0, axis=1)

    st.table(balance)

    # Totales y Utilidad
    t = balance.sum(numeric_only=True)
    utilidad_inv = t['Activo'] - t['Pasivo']
    utilidad_res = t['Ganancia'] - t['Pérdida']

    c1, c2 = st.columns(2)
    with c1:
        st.metric("Resultado por Inventario", f"${utilidad_inv:,.0f}")
    with c2:
        st.metric("Resultado por Ganancia/Pérdida", f"${utilidad_res:,.0f}")

    if utilidad_inv == utilidad_res:
        st.success("✅ ¡Balance Cuadrado!")
    else:
        st.warning("⚠️ El balance no cuadra. Revisa las clasificaciones.")

    # --- MÓDULO SII ---
    st.divider()
    col_f29, col_f22 = st.columns(2)

    with col_f29:
        st.subheader("📑 Simulador F29")
        iva_d = df[df['Cuenta'].str.contains("Débito", case=False)]['Haber'].sum()
        iva_c = df[df['Cuenta'].str.contains("Crédito", case=False)]['Debe'].sum()
        ingresos = t['Ganancia']
        ppm = round(ingresos * 0.01)
        total_pago = max(0, iva_d - iva_c) + ppm
        st.write(f"IVA a Pagar: ${max(0, iva_d - iva_c)}")
        st.write(f"PPM (1%): ${ppm}")
        st.markdown(f"**Total Tesorería: ${total_pago}**")

else:
    st.info("Aún no hay datos. Registra un asiento arriba.")

# Sidebar - Herramientas
with st.sidebar:
    st.header("Controles")
    if st.button("🗑️ Resetear Sistema"):
        st.session_state.movimientos = []
        st.rerun()

    if st.session_state.movimientos:
        csv = pd.DataFrame(st.session_state.movimientos).to_csv(index=False).encode('utf-8')
        st.download_button("📥 Descargar CSV", csv, "contabilidad.csv", "text/csv")

Overwriting app.py


In [23]:
!nohup streamlit run app.py --server.port 8501 &

nohup: appending output to 'nohup.out'


In [25]:
# 1. Instalación de herramientas
!pip install streamlit pyngrok -q

import os
import subprocess
from pyngrok import ngrok

# --- CONFIGURACIÓN DE SEGURIDAD ---
# Consigue tu token en https://dashboard.ngrok.com/get-started/your-authtoken
TOKEN = "3A5ylB0op0k2iL68de7ztNgBVh5_5Fb2KhvBNQQd5RR3yczLs"
ngrok.set_auth_token(TOKEN)

# 2. Creación del archivo app.py mejorado
with open("app.py", "w") as f:
    f.write("""
import streamlit as st
import pandas as pd
from datetime import datetime

st.set_page_config(page_title="Contabilidad Pro Elite", layout="wide")

# Estilos CSS para las Cuentas T
st.markdown('''
<style>
    .t-account-container {
        border: 2px solid #333;
        margin-bottom: 20px;
        background-color: white;
    }
    .t-header {
        background-color: #f0f2f6;
        text-align: center;
        font-weight: bold;
        border-bottom: 2px solid #333;
        padding: 5px;
        color: #1f1f1f;
    }
    .t-body {
        display: flex;
        min-height: 100px;
    }
    .t-side {
        width: 50%;
        padding: 10px;
    }
    .t-left {
        border-right: 2px solid #333;
        text-align: right;
        color: #198754;
    }
    .t-right {
        text-align: left;
        color: #dc3545;
    }
    .t-footer {
        border-top: 2px solid #333;
        display: flex;
        font-weight: bold;
        background-color: #f8f9fa;
    }
</style>
''', unsafe_allow_html=True)

if 'movimientos' not in st.session_state:
    st.session_state.movimientos = []

st.title("📊 Contabilidad Pro Elite v2026")

# --- ENTRADA DE DATOS ---
with st.expander("➕ Registrar Nuevo Asiento", expanded=True):
    col1, col2, col3, col4 = st.columns([1, 2, 2, 1])
    with col1: fecha = st.date_input("Fecha", datetime.now())
    with col2: cuenta = st.text_input("Cuenta", placeholder="Ej: Mercaderías")
    with col3: glosa = st.text_input("Glosa")
    with col4: clase = st.selectbox("Clasificación", ["Activo", "Pasivo", "Gasto", "Ingreso"])

    c_debe, c_haber = st.columns(2)
    debe = c_debe.number_input("Debe $", min_value=0, step=100)
    haber = c_haber.number_input("Haber $", min_value=0, step=100)

    if st.button("💾 Guardar Asiento", use_container_width=True):
        if cuenta:
            st.session_state.movimientos.append({
                "Fecha": str(fecha), "Cuenta": cuenta.upper(), "Glosa": glosa,
                "Debe": debe, "Haber": haber, "Clasificación": clase
            })
            st.success(f"Asiento en {cuenta} guardado")
        else:
            st.error("Debes indicar el nombre de la cuenta")

if st.session_state.movimientos:
    df = pd.DataFrame(st.session_state.movimientos)

    # --- LIBRO DIARIO ---
    st.divider()
    st.subheader("1. Libro Diario")
    st.dataframe(df, use_container_width=True)

    # --- LIBRO MAYOR (CUENTAS T) ---
    st.divider()
    st.subheader("2. Libro Mayor (Esquemas T)")

    cuentas = df['Cuenta'].unique()
    # Crear una cuadrícula para las cuentas T (3 por fila)
    rows = [cuentas[i:i + 3] for i in range(0, len(cuentas), 3)]

    for row_cuentas in rows:
        cols = st.columns(3)
        for idx, cta in enumerate(row_cuentas):
            df_cta = df[df['Cuenta'] == cta]
            sum_debe = df_cta['Debe'].sum()
            sum_haber = df_cta['Haber'].sum()

            # Construir la visualización T en HTML
            movs_debe = "<br>".join([str(v) for v in df_cta[df_cta['Debe'] > 0]['Debe']])
            movs_haber = "<br>".join([str(v) for v in df_cta[df_cta['Haber'] > 0]['Haber']])

            with cols[idx]:
                st.markdown(f'''
                <div class="t-account-container">
                    <div class="t-header">{cta}</div>
                    <div class="t-body">
                        <div class="t-side t-left">{movs_debe if movs_debe else "-"}</div>
                        <div class="t-side t-right">{movs_haber if movs_haber else "-"}</div>
                    </div>
                    <div class="t-footer">
                        <div class="t-side t-left">{sum_debe}</div>
                        <div class="t-side t-right">{sum_haber}</div>
                    </div>
                </div>
                ''', unsafe_allow_html=True)

    # --- BALANCE 8 COLUMNAS ---
    st.divider()
    st.subheader("3. Balance de 8 Columnas")
    bal = df.groupby(['Cuenta', 'Clasificación']).agg({'Debe':'sum', 'Haber':'sum'}).reset_index()

    # Cálculos de saldos
    bal['Deudor'] = (bal['Debe'] - bal['Haber']).apply(lambda x: x if x > 0 else 0)
    bal['Acreedor'] = (bal['Haber'] - bal['Debe']).apply(lambda x: x if x > 0 else 0)

    # Inventario y Resultados
    bal['Activo'] = bal.apply(lambda x: x['Deudor'] if x['Clasificación'] == 'Activo' else 0, axis=1)
    bal['Pasivo'] = bal.apply(lambda x: x['Acreedor'] if x['Clasificación'] == 'Pasivo' else 0, axis=1)
    bal['Pérdida'] = bal.apply(lambda x: x['Deudor'] if x['Clasificación'] == 'Gasto' else 0, axis=1)
    bal['Ganancia'] = bal.apply(lambda x: x['Acreedor'] if x['Clasificación'] == 'Ingreso' else 0, axis=1)

    st.dataframe(bal, use_container_width=True)

else:
    st.info("Ingresa tu primer asiento para generar los libros.")
""")

# 3. Lanzar el Túnel y Streamlit
try:
    ngrok.kill()
    public_url = ngrok.connect(8501, proto="http")
    print(f"\\n✅ APP LISTA! Haz clic aquí: {public_url.public_url}\\n")
    subprocess.Popen(["streamlit", "run", "app.py"])
except Exception as e:
    print(f"❌ Error: {e}")

\n✅ APP LISTA! Haz clic aquí: https://jonnie-dilapidated-aylin.ngrok-free.dev\n


In [28]:
# 1. Instalación de herramientas
!pip install streamlit pyngrok -q

import os
import subprocess
from pyngrok import ngrok

# --- CONFIGURACIÓN DE SEGURIDAD ---
TOKEN = "3A5ylB0op0k2iL68de7ztNgBVh5_5Fb2KhvBNQQd5RR3yczLs"
ngrok.set_auth_token(TOKEN)

# 2. Creación del archivo app.py evolucionado
with open("app.py", "w") as f:
    f.write("""
import streamlit as st
import pandas as pd
import numpy as np
from datetime import datetime

st.set_page_config(page_title="Sistema Contable Pro Elite - Chile", layout="wide")

# Estilos CSS
st.markdown('''
<style>
    .t-account { border: 2px solid #333; margin-bottom: 10px; background: white; }
    .t-header { background: #2c3e50; color: white; text-align: center; font-weight: bold; padding: 5px; }
    .t-body { display: flex; min-height: 60px; }
    .t-side { width: 50%; padding: 5px; font-size: 0.8rem; }
    .t-left { border-right: 2px solid #333; text-align: right; color: #198754; }
    .t-right { text-align: left; color: #dc3545; }
    .total-row { background-color: #f8f9fa; font-weight: bold; border-top: 2px solid #dee2e6; }
</style>
''', unsafe_allow_html=True)

if 'movimientos' not in st.session_state:
    st.session_state.movimientos = []

st.title("🇨🇱 Suite Contable Pro v2026")
st.subheader("Balance de 8 Columnas & Gestión SII")

# --- REGISTRO DE ASIENTOS ---
with st.sidebar:
    st.header("⚙️ Panel de Control")
    if st.button("🗑️ Borrar Todo"):
        st.session_state.movimientos = []
        st.rerun()

    st.divider()
    st.write("**Simulador de IVA**")
    monto_neto = st.number_input("Monto Neto $", min_value=0, step=1000)
    tipo_iva = st.selectbox("Tipo de Operación", ["Compra (Crédito)", "Venta (Débito)"])
    if st.button("Calcular IVA (19%)"):
        iva = round(monto_neto * 0.19)
        st.info(f"IVA: ${iva:,} | Total: ${monto_neto + iva:,}")

with st.expander("➕ Registrar Nuevo Movimiento", expanded=True):
    c1, c2, c3, c4 = st.columns([1, 2, 2, 1])
    fecha = c1.date_input("Fecha", datetime.now())
    cuenta = c2.text_input("Nombre de la Cuenta", placeholder="Ej: IVA Crédito Fiscal")
    glosa = c3.text_input("Glosa / Detalle")
    clase = c4.selectbox("Clasificación", ["Activo", "Pasivo", "Gasto", "Ingreso"])

    d1, d2 = st.columns(2)
    debe = d1.number_input("Debe $", min_value=0, step=100)
    haber = d2.number_input("Haber $", min_value=0, step=100)

    if st.button("💾 Registrar Asiento", use_container_width=True):
        if cuenta:
            st.session_state.movimientos.append({
                "Fecha": str(fecha), "Cuenta": cuenta.upper(), "Glosa": glosa,
                "Debe": debe, "Haber": haber, "Clase": clase
            })
            st.toast("Asiento registrado correctamente")
        else:
            st.error("Falta el nombre de la cuenta")

if st.session_state.movimientos:
    df = pd.DataFrame(st.session_state.movimientos)

    # --- LIBRO MAYOR (CUENTAS T) ---
    st.divider()
    st.subheader("1. Libro Mayor (Esquemas T)")
    ctas_unicas = df['Cuenta'].unique()
    cols_t = st.columns(4)
    for i, cta in enumerate(ctas_unicas):
        df_c = df[df['Cuenta'] == cta]
        with cols_t[i % 4]:
            st.markdown(f'''
            <div class="t-account">
                <div class="t-header">{cta}</div>
                <div class="t-body">
                    <div class="t-side t-left">{"<br>".join(df_c[df_c['Debe']>0]['Debe'].astype(str))}</div>
                    <div class="t-side t-right">{"<br>".join(df_c[df_c['Haber']>0]['Haber'].astype(str))}</div>
                </div>
            </div>
            ''', unsafe_allow_html=True)

    # --- BALANCE DE 8 COLUMNAS ---
    st.divider()
    st.subheader("2. Balance de 8 Columnas")

    # Agrupar por cuenta y clase
    bal = df.groupby(['Cuenta', 'Clase']).agg({'Debe':'sum', 'Haber':'sum'}).reset_index()

    # 1-2. Sumas (Debe, Haber) -> Ya están en bal
    # 3-4. Saldos (Deudor, Acreedor)
    bal['Deudor'] = (bal['Debe'] - bal['Haber']).apply(lambda x: x if x > 0 else 0)
    bal['Acreedor'] = (bal['Haber'] - bal['Debe']).apply(lambda x: x if x > 0 else 0)

    # 5-6. Inventario (Activo, Pasivo)
    bal['Activo'] = bal.apply(lambda x: x['Deudor'] if x['Clase'] == 'Activo' else 0, axis=1)
    bal['Pasivo'] = bal.apply(lambda x: x['Acreedor'] if x['Clase'] == 'Pasivo' else 0, axis=1)

    # 7-8. Resultado (Pérdida, Ganancia)
    bal['Pérdida'] = bal.apply(lambda x: x['Deudor'] if x['Clase'] == 'Gasto' else 0, axis=1)
    bal['Ganancia'] = bal.apply(lambda x: x['Acreedor'] if x['Clase'] == 'Ingreso' else 0, axis=1)

    # Mostrar tabla formateada
    st.table(bal.style.format(precision=0))

    # Totales del Balance
    t = bal.sum(numeric_only=True)
    utilidad_inv = t['Activo'] - t['Pasivo']
    utilidad_res = t['Ganancia'] - t['Pérdida']

    c1, c2, c3, c4 = st.columns(4)
    c1.metric("Total Activos", f"${t['Activo']:,}")
    c2.metric("Total Pasivos", f"${t['Pasivo']:,}")
    c3.metric("Total Pérdidas", f"${t['Pérdida']:,}")
    c4.metric("Total Ganancias", f"${t['Ganancia']:,}")

    if utilidad_inv == utilidad_res:
        st.success("✅ ¡Balance Cuadrado!")
    else:
        st.warning("⚠️ El balance no cuadra. Revisa las clasificaciones.")
""")

# 3. Lanzar el Túnel y Streamlit
try:
    ngrok.kill()
    public_url = ngrok.connect(8501, proto="http")
    print(f"\n✅ APP LISTA! Haz clic aquí: {public_url.public_url}\n")
    subprocess.Popen(["streamlit", "run", "app.py"])
except Exception as e:
    print(f"❌ Error: {e}")


✅ APP LISTA! Haz clic aquí: https://jonnie-dilapidated-aylin.ngrok-free.dev



In [29]:
# 1. Instalación de herramientas
!pip install streamlit pyngrok -q

import os
import subprocess
from pyngrok import ngrok

# --- CONFIGURACIÓN DE SEGURIDAD ---
# Pon tu token aquí. Consíguelo en: https://dashboard.ngrok.com/get-started/your-authtoken
TOKEN = "3A5ylB0op0k2iL68de7ztNgBVh5_5Fb2KhvBNQQd5RR3yczLs"
ngrok.set_auth_token(TOKEN)

# 2. Creación del archivo app.py (Versión con Libro Diario Detallado)
with open("app.py", "w") as f:
    f.write("""
import streamlit as st
import pandas as pd
from datetime import datetime

st.set_page_config(page_title="Sistema Contable Pro - Libro Diario", layout="wide")

# Estilos CSS para el diseño
st.markdown('''
<style>
    .t-account { border: 2px solid #333; margin-bottom: 15px; background: #fff; }
    .t-header { background: #1e3a8a; color: white; text-align: center; font-weight: bold; padding: 5px; }
    .t-body { display: flex; min-height: 50px; }
    .t-side { width: 50%; padding: 5px; font-size: 0.85rem; }
    .t-left { border-right: 2px solid #333; text-align: right; color: #059669; }
    .t-right { text-align: left; color: #dc2626; }
    .stDataFrame { border: 1px solid #e5e7eb; border-radius: 8px; }
</style>
''', unsafe_allow_html=True)

if 'movimientos' not in st.session_state:
    st.session_state.movimientos = []

st.title("📊 Gestión Contable Profesional")
st.subheader("Libro Diario, Cuentas T y Balance de 8 Columnas")

# --- SECCIÓN 1: INGRESO DE DATOS (ASIENTOS) ---
with st.container(border=True):
    st.write("### 📝 Registrar Nuevo Asiento")
    col1, col2, col3 = st.columns([1, 2, 2])

    with col1:
        fecha = st.date_input("Fecha", datetime.now())
    with col2:
        cuenta = st.text_input("Cuenta (Nombre)", placeholder="Ej: Banco Estado")
    with col3:
        glosa = st.text_input("Glosa (Descripción)", placeholder="Ej: Pago de arriendo mes marzo")

    col4, col5, col6 = st.columns([1, 1, 1])
    with col4:
        clase = st.selectbox("Clasificación", ["Activo", "Pasivo", "Gasto", "Ingreso"])
    with col5:
        debe = st.number_input("Debe ($)", min_value=0, step=500)
    with col6:
        haber = st.number_input("Haber ($)", min_value=0, step=500)

    if st.button("➕ Agregar al Libro Diario", use_container_width=True):
        if cuenta and (debe > 0 or haber > 0):
            st.session_state.movimientos.append({
                "Fecha": fecha.strftime("%d/%m/%Y"),
                "Cuenta": cuenta.upper(),
                "Glosa": glosa,
                "Debe": debe,
                "Haber": haber,
                "Clase": clase
            })
            st.success(f"Asiento registrado: {cuenta}")
        else:
            st.error("Por favor, ingresa el nombre de la cuenta y un monto en Debe o Haber.")

# --- SECCIÓN 2: EL LIBRO DIARIO (SOLICITADO) ---
if st.session_state.movimientos:
    df = pd.DataFrame(st.session_state.movimientos)

    st.divider()
    st.subheader("📑 1. Libro Diario Mensual")
    # Mostramos las columnas solicitadas: Fecha, Cuenta, Debe, Haber (y Glosa)
    st.dataframe(
        df[["Fecha", "Cuenta", "Glosa", "Debe", "Haber"]],
        use_container_width=True,
        hide_index=True
    )

    # Totales del Diario para verificar Partida Doble
    total_debe = df["Debe"].sum()
    total_haber = df["Haber"].sum()

    c_t1, c_t2 = st.columns(2)
    c_t1.metric("Total DEBE Diario", f"${total_debe:,}")
    c_t2.metric("Total HABER Diario", f"${total_haber:,}")

    if total_debe != total_haber:
        st.warning("⚠️ ¡Atención! El Libro Diario no suma lo mismo en Debe y Haber (Error de partida doble).")

    # --- SECCIÓN 3: LIBRO MAYOR (CUENTAS T) ---
    st.divider()
    st.subheader("📋 2. Libro Mayor (Esquemas T)")
    ctas = df['Cuenta'].unique()
    cols_t = st.columns(4)
    for i, cta in enumerate(ctas):
        df_c = df[df['Cuenta'] == cta]
        with cols_t[i % 4]:
            st.markdown(f'''
            <div class="t-account">
                <div class="t-header">{cta}</div>
                <div class="t-body">
                    <div class="t-side t-left">{"<br>".join(df_c[df_c['Debe']>0]['Debe'].astype(str))}</div>
                    <div class="t-side t-right">{"<br>".join(df_c[df_c['Haber']>0]['Haber'].astype(str))}</div>
                </div>
                <div style="border-top:1px solid #333; display:flex; font-weight:bold; font-size:0.8rem; background:#f9fafb;">
                    <div style="width:50%; text-align:right; padding:2px; border-right:1px solid #333;">{df_c['Debe'].sum()}</div>
                    <div style="width:50%; text-align:left; padding:2px;">{df_c['Haber'].sum()}</div>
                </div>
            </div>
            ''', unsafe_allow_html=True)

    # --- SECCIÓN 4: BALANCE DE 8 COLUMNAS ---
    st.divider()
    st.subheader("⚖️ 3. Balance de 8 Columnas")
    bal = df.groupby(['Cuenta', 'Clase']).agg({'Debe':'sum', 'Haber':'sum'}).reset_index()
    bal['Deudor'] = (bal['Debe'] - bal['Haber']).apply(lambda x: x if x > 0 else 0)
    bal['Acreedor'] = (bal['Haber'] - bal['Debe']).apply(lambda x: x if x > 0 else 0)
    bal['Activo'] = bal.apply(lambda x: x['Deudor'] if x['Clase'] == 'Activo' else 0, axis=1)
    bal['Pasivo'] = bal.apply(lambda x: x['Acreedor'] if x['Clase'] == 'Pasivo' else 0, axis=1)
    bal['Pérdida'] = bal.apply(lambda x: x['Deudor'] if x['Clase'] == 'Gasto' else 0, axis=1)
    bal['Ganancia'] = bal.apply(lambda x: x['Acreedor'] if x['Clase'] == 'Ingreso' else 0, axis=1)

    st.table(bal.style.format({
        "Debe": "{:,.0f}", "Haber": "{:,.0f}", "Deudor": "{:,.0f}",
        "Acreedor": "{:,.0f}", "Activo": "{:,.0f}", "Pasivo": "{:,.0f}",
        "Pérdida": "{:,.0f}", "Ganancia": "{:,.0f}"
    }))

    # --- IVA MENSUAL CHILE ---
    st.divider()
    st.subheader("🇨🇱 Resumen IVA Mensual (SII)")
    iva_c = df[df['Cuenta'].str.contains("CRÉDITO", na=False)]['Debe'].sum()
    iva_d = df[df['Cuenta'].str.contains("DÉBITO", na=False)]['Haber'].sum()
    st.info(f"IVA Crédito: ${iva_c:,} | IVA Débito: ${iva_d:,} | **Neto F29: ${max(0, iva_d - iva_c):,}**")

else:
    st.info("💡 Comienza ingresando un asiento arriba para generar los libros contables.")

with st.sidebar:
    st.header("Opciones")
    if st.button("🗑️ Reiniciar Todo"):
        st.session_state.movimientos = []
        st.rerun()
    if st.session_state.movimientos:
        csv = pd.DataFrame(st.session_state.movimientos).to_csv(index=False)
        st.download_button("📥 Descargar Libro Diario", csv, "diario.csv", "text/csv")
""")

# 3. Lanzar Túnel y App
try:
    ngrok.kill()
    public_url = ngrok.connect(8501, proto="http")
    print(f"\\n✅ APP FUNCIONANDO! Haz clic aquí: {public_url.public_url}\\n")
    subprocess.Popen(["streamlit", "run", "app.py"])
except Exception as e:
    print(f"❌ Error: {e}")

\n✅ APP FUNCIONANDO! Haz clic aquí: https://jonnie-dilapidated-aylin.ngrok-free.dev\n


In [30]:
# 1. Instalación de herramientas
!pip install streamlit pyngrok -q

import os
import subprocess
from pyngrok import ngrok

# --- CONFIGURACIÓN DE SEGURIDAD ---
TOKEN = "3A5ylB0op0k2iL68de7ztNgBVh5_5Fb2KhvBNQQd5RR3yczLs"
ngrok.set_auth_token(TOKEN)

# 2. Creación del archivo app.py
with open("app.py", "w") as f:
    f.write("""
import streamlit as st
import pandas as pd
from datetime import datetime

st.set_page_config(page_title="Contabilidad Pro + IVA Chile", layout="wide")

# Estilos CSS
st.markdown('''
<style>
    .t-account { border: 2px solid #333; margin-bottom: 10px; background: white; }
    .t-header { background: #1e3a8a; color: white; text-align: center; font-weight: bold; padding: 5px; }
    .t-body { display: flex; min-height: 50px; }
    .t-side { width: 50%; padding: 5px; font-size: 0.8rem; }
    .t-left { border-right: 2px solid #333; text-align: right; color: #16a34a; }
    .t-right { text-align: left; color: #dc2626; }
    .balance-table { font-size: 0.85rem !important; }
</style>
''', unsafe_allow_html=True)

if 'movimientos' not in st.session_state:
    st.session_state.movimientos = []

st.title("🇨🇱 Sistema Contable Profesional v2026")
st.subheader("Libro Diario, Cuentas T, Balance de 8 Columnas e IVA")

# --- MÓDULO DE INGRESO CON CÁLCULO DE IVA ---
with st.container(border=True):
    st.markdown("### 📝 Registro de Asientos y Cálculo de IVA")

    col1, col2, col3 = st.columns([1, 2, 2])
    with col1: fecha = st.date_input("Fecha", datetime.now())
    with col2: cuenta = st.text_input("Cuenta", placeholder="Ej: Mercaderías o Ventas")
    with col3: glosa = st.text_input("Glosa", placeholder="Ej: Factura N°123")

    col4, col5, col6 = st.columns([1, 1, 1])
    with col4: clase = st.selectbox("Clase", ["Activo", "Pasivo", "Gasto", "Ingreso"])
    with col5: debe = st.number_input("Debe $", min_value=0, step=100)
    with col6: haber = st.number_input("Haber $", min_value=0, step=100)

    # BOTONES DE ACCIÓN
    c_btn1, c_btn2 = st.columns(2)

    if c_btn1.button("💾 Guardar Asiento Simple", use_container_width=True):
        if cuenta:
            st.session_state.movimientos.append({
                "Fecha": fecha.strftime("%d/%m/%Y"), "Cuenta": cuenta.upper(),
                "Glosa": glosa, "Debe": debe, "Haber": haber, "Clase": clase
            })
            st.success("Guardado.")
        else: st.error("Falta cuenta")

    if c_btn2.button("✨ Guardar + Calcular IVA (19%)", use_container_width=True, type="primary"):
        if cuenta and (debe > 0 or haber > 0):
            # 1. Guardar el asiento principal (Neto)
            st.session_state.movimientos.append({
                "Fecha": fecha.strftime("%d/%m/%Y"), "Cuenta": cuenta.upper(),
                "Glosa": glosa, "Debe": debe, "Haber": haber, "Clase": clase
            })
            # 2. Calcular y guardar el IVA
            monto_iva = round((debe if debe > 0 else haber) * 0.19)
            es_compra = debe > 0

            st.session_state.movimientos.append({
                "Fecha": fecha.strftime("%d/%m/%Y"),
                "Cuenta": "IVA CRÉDITO FISCAL" if es_compra else "IVA DÉBITO FISCAL",
                "Glosa": f"IVA de: {glosa}",
                "Debe": monto_iva if es_compra else 0,
                "Haber": 0 if es_compra else monto_iva,
                "Clase": "Activo" if es_compra else "Pasivo"
            })
            st.success(f"Asiento + IVA (${monto_iva:,}) registrados.")
        else: st.error("Ingresa un monto para calcular IVA")

if st.session_state.movimientos:
    df = pd.DataFrame(st.session_state.movimientos)

    # --- 1. LIBRO DIARIO ---
    st.divider()
    st.subheader("📑 1. Libro Diario")
    st.dataframe(df[["Fecha", "Cuenta", "Glosa", "Debe", "Haber"]], use_container_width=True, hide_index=True)

    # --- 2. LIBRO MAYOR (CUENTAS T) ---
    st.divider()
    st.subheader("📋 2. Libro Mayor (Cuentas T)")
    ctas = df['Cuenta'].unique()
    cols_t = st.columns(4)
    for i, cta in enumerate(ctas):
        df_c = df[df['Cuenta'] == cta]
        with cols_t[i % 4]:
            st.markdown(f'''
            <div class="t-account">
                <div class="t-header">{cta}</div>
                <div class="t-body">
                    <div class="t-side t-left">{"<br>".join(df_c[df_c['Debe']>0]['Debe'].astype(str))}</div>
                    <div class="t-side t-right">{"<br>".join(df_c[df_c['Haber']>0]['Haber'].astype(str))}</div>
                </div>
                <div style="border-top:1px solid #333; display:flex; font-weight:bold; font-size:0.8rem; background:#f3f4f6;">
                    <div style="width:50%; text-align:right; padding:2px; border-right:1px solid #333;">{df_c['Debe'].sum():,}</div>
                    <div style="width:50%; text-align:left; padding:2px;">{df_c['Haber'].sum():,}</div>
                </div>
            </div>
            ''', unsafe_allow_html=True)

    # --- 3. BALANCE DE 8 COLUMNAS ---
    st.divider()
    st.subheader("⚖️ 3. Balance de 8 Columnas")
    bal = df.groupby(['Cuenta', 'Clase']).agg({'Debe':'sum', 'Haber':'sum'}).reset_index()
    bal['Deudor'] = (bal['Debe'] - bal['Haber']).apply(lambda x: x if x > 0 else 0)
    bal['Acreedor'] = (bal['Haber'] - bal['Debe']).apply(lambda x: x if x > 0 else 0)
    bal['Activo'] = bal.apply(lambda x: x['Deudor'] if x['Clase'] == 'Activo' else 0, axis=1)
    bal['Pasivo'] = bal.apply(lambda x: x['Acreedor'] if x['Clase'] == 'Pasivo' else 0, axis=1)
    bal['Pérdida'] = bal.apply(lambda x: x['Deudor'] if x['Clase'] == 'Gasto' else 0, axis=1)
    bal['Ganancia'] = bal.apply(lambda x: x['Acreedor'] if x['Clase'] == 'Ingreso' else 0, axis=1)

    st.table(bal.style.format(precision=0))

    # Totales Finales
    t = bal.sum(numeric_only=True)
    st.markdown(f"**Resultado del Ejercicio:** Balance {(t['Activo']-t['Pasivo']):,} | Estado R.: {(t['Ganancia']-t['Pérdida']):,}")

    # --- 4. RESUMEN IVA MENSUAL ---
    st.divider()
    st.subheader("🏢 Resumen Tributario Mensual (SII)")
    iva_c = df[df['Cuenta'] == "IVA CRÉDITO FISCAL"]['Debe'].sum()
    iva_d = df[df['Cuenta'] == "IVA DÉBITO FISCAL"]['Haber'].sum()
    neto_iva = iva_d - iva_c

    c1, c2, c3 = st.columns(3)
    c1.metric("Total IVA Crédito (Compras)", f"${iva_c:,}")
    c2.metric("Total IVA Débito (Ventas)", f"${iva_d:,}")
    c3.metric("IVA a Pagar / Remanente", f"${neto_iva:,}", delta=-neto_iva, delta_color="inverse")

else:
    st.info("Ingresa un asiento para comenzar.")

with st.sidebar:
    if st.button("🗑️ Reiniciar"):
        st.session_state.movimientos = []
        st.rerun()
""")

# 3. Lanzamiento
try:
    ngrok.kill()
    public_url = ngrok.connect(8501, proto="http")
    print(f"\\n✅ LINK DE TU WEB: {public_url.public_url}\\n")
    subprocess.Popen(["streamlit", "run", "app.py"])
except Exception as e:
    print(f"❌ Error: {e}")

\n✅ LINK DE TU WEB: https://jonnie-dilapidated-aylin.ngrok-free.dev\n


In [32]:
# 1. Instalación de herramientas
!pip install streamlit pyngrok -q

import os
import subprocess
from pyngrok import ngrok

# --- CONFIGURACIÓN DE SEGURIDAD ---
TOKEN = "3A5ylB0op0k2iL68de7ztNgBVh5_5Fb2KhvBNQQd5RR3yczLs"
ngrok.set_auth_token(TOKEN)

# 2. Creación del archivo app.py
with open("app.py", "w") as f:
    f.write("""
import streamlit as st
import pandas as pd
from datetime import datetime
import calendar

st.set_page_config(page_title="Contabilidad Pro Elite - Cierre Mensual", layout="wide")

# Estilos CSS
st.markdown('''
<style>
    .t-account { border: 2px solid #333; margin-bottom: 10px; background: white; }
    .t-header { background: #1e3a8a; color: white; text-align: center; font-weight: bold; padding: 5px; }
    .t-body { display: flex; min-height: 50px; }
    .t-side { width: 50%; padding: 5px; font-size: 0.8rem; }
    .t-left { border-right: 2px solid #333; text-align: right; color: #16a34a; }
    .t-right { text-align: left; color: #dc2626; }
    .stMetric { background-color: #f8f9fa; padding: 10px; border-radius: 5px; }
</style>
''', unsafe_allow_html=True)

# Inicialización de estados
if 'movimientos' not in st.session_state:
    st.session_state.movimientos = []
if 'cierres' not in st.session_state:
    st.session_state.cierres = []

st.title("🇨🇱 Suite Contable Full v2026")

# --- SIDEBAR: FILTROS DE LIBRO MENSUAL Y CIERRE ---
with st.sidebar:
    st.header("📅 Libro Mensual")
    meses_nombres = ["Enero", "Febrero", "Marzo", "Abril", "Mayo", "Junio",
                     "Julio", "Agosto", "Septiembre", "Octubre", "Noviembre", "Diciembre"]
    mes_selec = st.selectbox("Seleccionar Mes para Reportes", meses_nombres, index=datetime.now().month - 1)
    anio_selec = st.number_input("Año", min_value=2024, max_value=2030, value=datetime.now().year)

    st.divider()
    st.header("🏁 Cierre Mensual")
    if st.button("🚀 Realizar Cierre de Resultados"):
        # Lógica de cierre: Centralizar Ingresos y Gastos
        df_all = pd.DataFrame(st.session_state.movimientos)
        if not df_all.empty:
            ingresos = df_all[df_all['Clase'] == 'Ingreso']['Haber'].sum() - df_all[df_all['Clase'] == 'Ingreso']['Debe'].sum()
            gastos = df_all[df_all['Clase'] == 'Gasto']['Debe'].sum() - df_all[df_all['Clase'] == 'Gasto']['Haber'].sum()
            utilidad = ingresos - gastos

            # Crear asiento de cierre
            cierre_fecha = f"{anio_selec}-{meses_nombres.index(mes_selec)+1:02d}-28"
            st.session_state.movimientos.append({
                "Fecha": cierre_fecha, "Cuenta": "CIERRE DE RESULTADOS",
                "Glosa": f"Cierre mensual {mes_selec}", "Debe": ingresos, "Haber": gastos, "Clase": "Pasivo"
            })
            st.session_state.movimientos.append({
                "Fecha": cierre_fecha, "Cuenta": "UTILIDAD DEL EJERCICIO",
                "Glosa": f"Utilidad mes {mes_selec}", "Debe": 0, "Haber": utilidad, "Clase": "Pasivo"
            })
            st.success("¡Cierre efectuado! Las cuentas de resultado se han saldado.")

    st.divider()
    if st.button("🗑️ Resetear Sistema"):
        st.session_state.movimientos = []
        st.rerun()

# --- FORMULARIO DE INGRESO (CON IVA) ---
with st.expander("➕ Registro de Asientos (Auto-IVA 19%)", expanded=True):
    col1, col2, col3 = st.columns([1, 2, 2])
    fecha = col1.date_input("Fecha", datetime.now())
    cuenta = col2.text_input("Cuenta")
    glosa = col3.text_input("Glosa")

    col4, col5, col6 = st.columns([1, 1, 1])
    clase = col4.selectbox("Clasificación", ["Activo", "Pasivo", "Gasto", "Ingreso"])
    debe = col5.number_input("Debe $", min_value=0, step=100)
    haber = col6.number_input("Haber $", min_value=0, step=100)

    c1, c2 = st.columns(2)
    if c1.button("💾 Guardar Simple", use_container_width=True):
        st.session_state.movimientos.append({
            "Fecha": str(fecha), "Cuenta": cuenta.upper(), "Glosa": glosa,
            "Debe": debe, "Haber": haber, "Clase": clase
        })

    if c2.button("✨ Guardar + IVA (19%)", type="primary", use_container_width=True):
        st.session_state.movimientos.append({
            "Fecha": str(fecha), "Cuenta": cuenta.upper(), "Glosa": glosa,
            "Debe": debe, "Haber": haber, "Clase": clase
        })
        monto_iva = round((debe if debe > 0 else haber) * 0.19)
        es_compra = debe > 0
        st.session_state.movimientos.append({
            "Fecha": str(fecha), "Cuenta": "IVA CRÉDITO FISCAL" if es_compra else "IVA DÉBITO FISCAL",
            "Glosa": f"IVA de {glosa}", "Debe": monto_iva if es_compra else 0,
            "Haber": 0 if es_compra else monto_iva, "Clase": "Activo" if es_compra else "Pasivo"
        })

# --- PROCESAMIENTO DE DATOS ---
if st.session_state.movimientos:
    df = pd.DataFrame(st.session_state.movimientos)
    df['Fecha'] = pd.to_datetime(df['Fecha'])

    # Filtro de Libro Mensual
    mes_num = meses_nombres.index(mes_selec) + 1
    df_mes = df[(df['Fecha'].dt.month == mes_num) & (df['Fecha'].dt.year == anio_selec)]

    if df_mes.empty:
        st.warning(f"No hay movimientos registrados para {mes_selec} de {anio_selec}")
    else:
        # 1. LIBRO DIARIO
        st.subheader(f"📑 Libro Diario: {mes_selec} {anio_selec}")
        st.dataframe(df_mes, use_container_width=True)

        # 2. LIBRO MAYOR (CUENTAS T)
        st.divider()
        st.subheader("📋 Libro Mayor (Cuentas T)")
        ctas = df_mes['Cuenta'].unique()
        cols = st.columns(4)
        for i, cta in enumerate(ctas):
            d_cta = df_mes[df_mes['Cuenta'] == cta]
            with cols[i % 4]:
                st.markdown(f'''
                <div class="t-account">
                    <div class="t-header">{cta}</div>
                    <div class="t-body">
                        <div class="t-side t-left">{"<br>".join(d_cta[d_cta['Debe']>0]['Debe'].astype(str))}</div>
                        <div class="t-side t-right">{"<br>".join(d_cta[d_cta['Haber']>0]['Haber'].astype(str))}</div>
                    </div>
                    <div style="border-top:1px solid #333; display:flex; font-weight:bold; padding:2px; font-size:0.7rem;">
                        <div style="width:50%; text-align:right; border-right:1px solid #333; padding-right:5px;">{d_cta['Debe'].sum():,}</div>
                        <div style="width:50%; text-align:left; padding-left:5px;">{d_cta['Haber'].sum():,}</div>
                    </div>
                </div>''', unsafe_allow_html=True)

        # 3. BALANCE DE 8 COLUMNAS
        st.divider()
        st.subheader("⚖️ Balance de Comprobación y Saldos (8 Columnas)")
        bal = df_mes.groupby(['Cuenta', 'Clase']).agg({'Debe':'sum', 'Haber':'sum'}).reset_index()
        bal['Deudor'] = (bal['Debe'] - bal['Haber']).apply(lambda x: x if x > 0 else 0)
        bal['Acreedor'] = (bal['Haber'] - bal['Debe']).apply(lambda x: x if x > 0 else 0)
        bal['Activo'] = bal.apply(lambda x: x['Deudor'] if x['Clase'] == 'Activo' else 0, axis=1)
        bal['Pasivo'] = bal.apply(lambda x: x['Acreedor'] if x['Clase'] == 'Pasivo' else 0, axis=1)
        bal['Pérdida'] = bal.apply(lambda x: x['Deudor'] if x['Clase'] == 'Gasto' else 0, axis=1)
        bal['Ganancia'] = bal.apply(lambda x: x['Acreedor'] if x['Clase'] == 'Ingreso' else 0, axis=1)
        st.table(bal.style.format(precision=0))

        # 4. RESUMEN IVA
        st.divider()
        col_iva1, col_iva2 = st.columns(2)
        iva_c = df_mes[df_mes['Cuenta'] == "IVA CRÉDITO FISCAL"]['Debe'].sum()
        iva_d = df_mes[df_mes['Cuenta'] == "IVA DÉBITO FISCAL"]['Haber'].sum()
        col_iva1.metric("IVA Crédito (Compras)", f"${iva_c:,}")
        col_iva2.metric("IVA Débito (Ventas)", f"${iva_d:,}")
        st.info(f"**Impuesto Neto a Pagar (F29): ${max(0, iva_d - iva_c):,}**")

else:
    st.info("Ingresa datos para activar los Libros Contables.")
""")

# 3. Lanzar Túnel y App
try:
    ngrok.kill()
    public_url = ngrok.connect(8501, proto="http")
    print(f"\\n✅ ACCESO ONLINE: {public_url.public_url}\\n")
    subprocess.Popen(["streamlit", "run", "app.py"])
except Exception as e:
    print(f"❌ Error: {e}")

\n✅ ACCESO ONLINE: https://jonnie-dilapidated-aylin.ngrok-free.dev\n


In [33]:
# 1. Instalación de herramientas (Agregamos python-docx para Word)
!pip install streamlit pyngrok python-docx -q

import os
import subprocess
from pyngrok import ngrok

# --- CONFIGURACIÓN DE SEGURIDAD ---
TOKEN = "3A5ylB0op0k2iL68de7ztNgBVh5_5Fb2KhvBNQQd5RR3yczLs"
ngrok.set_auth_token(TOKEN)

# 2. Creación del archivo app.py
with open("app.py", "w") as f:
    f.write("""
import streamlit as st
import pandas as pd
from datetime import datetime
from docx import Document
from io import BytesIO

st.set_page_config(page_title="Contabilidad Pro Elite - Exportación", layout="wide")

# Estilos CSS (Incluye reglas para impresión)
st.markdown('''
<style>
    .t-account { border: 2px solid #333; margin-bottom: 10px; background: white; }
    .t-header { background: #1e3a8a; color: white; text-align: center; font-weight: bold; padding: 5px; }
    .t-body { display: flex; min-height: 50px; }
    .t-side { width: 50%; padding: 5px; font-size: 0.8rem; }
    .t-left { border-right: 2px solid #333; text-align: right; color: #16a34a; }
    .t-right { text-align: left; color: #dc2626; }

    @media print {
        .stButton, .stDownloadButton, .no-print { display: none !important; }
        .print-only { display: block !important; }
    }
</style>
''', unsafe_allow_html=True)

if 'movimientos' not in st.session_state:
    st.session_state.movimientos = []

st.title("📊 Suite Contable v2026: Gestión y Reportes")

# --- FUNCIONES DE EXPORTACIÓN ---
def generar_word(df_balance, mes):
    doc = Document()
    doc.add_heading(f'Balance de 8 Columnas - Periodo {mes}', 0)
    table = doc.add_table(rows=1, cols=len(df_balance.columns))
    hdr_cells = table.rows[0].cells
    for i, col in enumerate(df_balance.columns):
        hdr_cells[i].text = col
    for _, row in df_balance.iterrows():
        row_cells = table.add_row().cells
        for i, val in enumerate(row):
            row_cells[i].text = str(val)
    bio = BytesIO()
    doc.save(bio)
    return bio.getvalue()

# --- SIDEBAR Y CONTROLES ---
with st.sidebar:
    st.header("⚙️ Exportar Reportes")
    if st.session_state.movimientos:
        df_full = pd.DataFrame(st.session_state.movimientos)

        # EXCEL
        towrite = BytesIO()
        df_full.to_excel(towrite, index=False, engine='openpyxl')
        st.download_button("📥 Descargar Excel", towrite.getvalue(), "contabilidad_completa.xlsx", "application/vnd.openxmlformats-officedocument.spreadsheetml.sheet")

        # IMPRIMIR (JS)
        if st.button("🖨️ Preparar para Imprimir"):
            st.markdown('<script>window.print();</script>', unsafe_allow_html=True)
            st.info("Usa Ctrl+P para imprimir si el diálogo no abre.")

    st.divider()
    if st.button("🗑️ Resetear Datos"):
        st.session_state.movimientos = []
        st.rerun()

# --- FORMULARIO ---
with st.expander("➕ Registro de Asientos (IVA 19%)", expanded=False):
    c1, c2, c3 = st.columns([1, 2, 2])
    fecha = c1.date_input("Fecha", datetime.now())
    cuenta = c2.text_input("Cuenta")
    glosa = c3.text_input("Glosa")
    clase = st.selectbox("Clasificación", ["Activo", "Pasivo", "Gasto", "Ingreso"])
    d, h = st.columns(2)
    debe = d.number_input("Debe $", min_value=0)
    haber = h.number_input("Haber $", min_value=0)

    if st.button("✨ Guardar con IVA Automático", type="primary", use_container_width=True):
        st.session_state.movimientos.append({"Fecha": str(fecha), "Cuenta": cuenta.upper(), "Glosa": glosa, "Debe": debe, "Haber": haber, "Clase": clase})
        iva = round((debe if debe > 0 else haber) * 0.19)
        es_compra = debe > 0
        st.session_state.movimientos.append({
            "Fecha": str(fecha), "Cuenta": "IVA CRÉDITO" if es_compra else "IVA DÉBITO",
            "Glosa": f"IVA de {glosa}", "Debe": iva if es_compra else 0, "Haber": 0 if es_compra else iva, "Clase": "Activo" if es_compra else "Pasivo"
        })

# --- VISUALIZACIÓN ---
if st.session_state.movimientos:
    df = pd.DataFrame(st.session_state.movimientos)

    tab1, tab2 = st.tabs(["📖 Contabilidad Mensual", "📑 Formularios SII"])

    with tab1:
        st.subheader("Balance de 8 Columnas")
        bal = df.groupby(['Cuenta', 'Clase']).agg({'Debe':'sum', 'Haber':'sum'}).reset_index()
        bal['Deudor'] = (bal['Debe'] - bal['Haber']).clip(lower=0)
        bal['Acreedor'] = (bal['Haber'] - bal['Debe']).clip(lower=0)
        bal['Activo'] = bal.apply(lambda x: x['Deudor'] if x['Clase'] == 'Activo' else 0, axis=1)
        bal['Pasivo'] = bal.apply(lambda x: x['Acreedor'] if x['Clase'] == 'Pasivo' else 0, axis=1)
        bal['Pérdida'] = bal.apply(lambda x: x['Deudor'] if x['Clase'] == 'Gasto' else 0, axis=1)
        bal['Ganancia'] = bal.apply(lambda x: x['Acreedor'] if x['Clase'] == 'Ingreso' else 0, axis=1)

        st.table(bal)

        # BOTÓN WORD
        word_data = generar_word(bal, "Actual")
        st.download_button("📝 Descargar Balance en Word", word_data, "Balance_8_Columnas.docx", "application/vnd.openxmlformats-officedocument.wordprocessingml.document")

    with tab2:
        st.subheader("Resumen F29 y F22")
        iva_c = df[df['Cuenta'] == "IVA CRÉDITO"]['Debe'].sum()
        iva_d = df[df['Cuenta'] == "IVA DÉBITO"]['Haber'].sum()
        st.warning(f"IVA Neto a Pagar: ${max(0, iva_d - iva_c):,}")
        st.info(f"Base Imponible F22: ${max(0, bal['Ganancia'].sum() - bal['Pérdida'].sum()):,}")

else:
    st.info("Ingresa datos para activar reportes.")
""")

# 3. Lanzamiento
try:
    ngrok.kill()
    public_url = ngrok.connect(8501, proto="http")
    print(f"\\n✅ SISTEMA LISTO: {public_url.public_url}\\n")
    subprocess.Popen(["streamlit", "run", "app.py"])
except Exception as e:
    print(f"❌ Error: {e}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 7.7 MB/s eta 0:00:00
\n✅ SISTEMA LISTO: https://jonnie-dilapidated-aylin.ngrok-free.dev\n


In [34]:
# 1. Instalación de herramientas
!pip install streamlit pyngrok python-docx openpyxl -q

import os
import subprocess
from pyngrok import ngrok

# --- CONFIGURACIÓN DE SEGURIDAD ---
TOKEN = "3A5ylB0op0k2iL68de7ztNgBVh5_5Fb2KhvBNQQd5RR3yczLs"
ngrok.set_auth_token(TOKEN)

# 2. Creación del archivo app.py evolucionado
with open("app.py", "w") as f:
    f.write("""
import streamlit as st
import pandas as pd
from datetime import datetime
from docx import Document
from io import BytesIO

st.set_page_config(page_title="Contabilidad Pro + Exportación", layout="wide")

# Estilos CSS Profesionales y Reglas de Impresión
st.markdown('''
<style>
    .t-account { border: 2px solid #333; margin-bottom: 10px; background: white; border-radius: 4px; }
    .t-header { background: #1e3a8a; color: white; text-align: center; font-weight: bold; padding: 5px; }
    .t-body { display: flex; min-height: 50px; }
    .t-side { width: 50%; padding: 5px; font-size: 0.8rem; }
    .t-left { border-right: 2px solid #333; text-align: right; color: #16a34a; }
    .t-right { text-align: left; color: #dc2626; }

    /* Optimización para impresión */
    @media print {
        .stButton, .stDownloadButton, .no-print, [data-testid="stSidebar"] {
            display: none !important;
        }
        .main { background-color: white !important; }
    }
</style>
''', unsafe_allow_html=True)

# Inicialización
if 'movimientos' not in st.session_state:
    st.session_state.movimientos = []

# --- FUNCIONES DE EXPORTACIÓN ---
def to_word(df_balance, titulo):
    doc = Document()
    doc.add_heading(titulo, 0)
    table = doc.add_table(rows=1, cols=len(df_balance.columns))
    table.style = 'Table Grid'
    hdr_cells = table.rows[0].cells
    for i, col in enumerate(df_balance.columns):
        hdr_cells[i].text = col
    for _, row in df_balance.iterrows():
        row_cells = table.add_row().cells
        for i, val in enumerate(row):
            row_cells[i].text = str(val)
    buffer = BytesIO()
    doc.save(buffer)
    return buffer.getvalue()

def to_excel(df):
    output = BytesIO()
    with pd.ExcelWriter(output, engine='openpyxl') as writer:
        df.to_excel(writer, index=False, sheet_name='Libro_Diario')
    return output.getvalue()

st.title("📊 Contabilidad 360: Chile v2026")

# --- SIDEBAR: GESTIÓN Y EXPORTACIÓN ---
with st.sidebar:
    st.header("⚙️ Herramientas de Reporte")
    if st.session_state.movimientos:
        df_export = pd.DataFrame(st.session_state.movimientos)

        # EXPORTAR EXCEL
        st.download_button(
            label="📥 Descargar Libro Diario (Excel)",
            data=to_excel(df_export),
            file_name=f"libro_diario_{datetime.now().strftime('%Y%m%d')}.xlsx",
            mime="application/vnd.openxmlformats-officedocument.spreadsheetml.sheet",
            use_container_width=True
        )

        # IMPRIMIR
        if st.button("🖨️ Imprimir Reporte Actual", use_container_width=True):
            st.markdown('<script>window.print();</script>', unsafe_allow_html=True)
            st.info("💡 Si el diálogo no abre, presiona Ctrl+P")

    st.divider()
    if st.button("🗑️ Resetear Datos"):
        st.session_state.movimientos = []
        st.rerun()

# --- ENTRADA DE DATOS ---
with st.expander("➕ Registro de Asientos (Auto-IVA 19%)"):
    c1, c2, c3 = st.columns([1, 2, 2])
    fecha = c1.date_input("Fecha", datetime.now())
    cuenta = c2.text_input("Cuenta")
    glosa = c3.text_input("Glosa")

    c4, c5, c6 = st.columns([1, 1, 1])
    clase = c4.selectbox("Clase", ["Activo", "Pasivo", "Gasto", "Ingreso"])
    debe = c5.number_input("Debe $", min_value=0)
    haber = c6.number_input("Haber $", min_value=0)

    if st.button("💾 Guardar con IVA", type="primary", use_container_width=True):
        if cuenta:
            st.session_state.movimientos.append({"Fecha": str(fecha), "Cuenta": cuenta.upper(), "Glosa": glosa, "Debe": debe, "Haber": haber, "Clase": clase})
            monto_iva = round((debe if debe > 0 else haber) * 0.19)
            es_compra = debe > 0
            st.session_state.movimientos.append({
                "Fecha": str(fecha), "Cuenta": "IVA CRÉDITO FISCAL" if es_compra else "IVA DÉBITO FISCAL",
                "Glosa": f"IVA de {glosa}", "Debe": monto_iva if es_compra else 0, "Haber": 0 if es_compra else monto_iva, "Clase": "Activo" if es_compra else "Pasivo"
            })
            st.rerun()

# --- REPORTES ---
if st.session_state.movimientos:
    df = pd.DataFrame(st.session_state.movimientos)
    tab1, tab2, tab3 = st.tabs(["📖 Libros y Balance", "📑 F29 Mensual", "🏦 F22 Anual"])

    with tab1:
        st.subheader("Balance de 8 Columnas")
        bal = df.groupby(['Cuenta', 'Clase']).agg({'Debe':'sum', 'Haber':'sum'}).reset_index()
        bal['Deudor'] = (bal['Debe'] - bal['Haber']).clip(lower=0)
        bal['Acreedor'] = (bal['Haber'] - bal['Debe']).clip(lower=0)
        bal['Activo'] = bal.apply(lambda x: x['Deudor'] if x['Clase'] == 'Activo' else 0, axis=1)
        bal['Pasivo'] = bal.apply(lambda x: x['Acreedor'] if x['Clase'] == 'Pasivo' else 0, axis=1)
        bal['Pérdida'] = bal.apply(lambda x: x['Deudor'] if x['Clase'] == 'Gasto' else 0, axis=1)
        bal['Ganancia'] = bal.apply(lambda x: x['Acreedor'] if x['Clase'] == 'Ingreso' else 0, axis=1)

        st.table(bal)

        # EXPORTAR WORD
        word_data = to_word(bal, f"Balance de 8 Columnas - {datetime.now().strftime('%d/%m/%Y')}")
        st.download_button(
            label="📝 Guardar Balance en Word",
            data=word_data,
            file_name="Balance_8_Columnas.docx",
            mime="application/vnd.openxmlformats-officedocument.wordprocessingml.document"
        )

        st.divider()
        st.subheader("Esquemas T")
        ctas = df['Cuenta'].unique()
        cols_t = st.columns(4)
        for i, cta in enumerate(ctas):
            d_cta = df[df['Cuenta'] == cta]
            with cols_t[i % 4]:
                st.markdown(f'<div class="t-account"><div class="t-header">{cta}</div><div class="t-body"><div class="t-side t-left">{"<br>".join(d_cta[d_cta["Debe"]>0]["Debe"].astype(str))}</div><div class="t-side t-right">{"<br>".join(d_cta[d_cta["Haber"]>0]["Haber"].astype(str))}</div></div></div>', unsafe_allow_html=True)

    with tab2:
        # Lógica F29 (Consolidada)
        iva_c = df[df['Cuenta'] == "IVA CRÉDITO FISCAL"]['Debe'].sum()
        iva_d = df[df['Cuenta'] == "IVA DÉBITO FISCAL"]['Haber'].sum()
        st.metric("Impuesto F29 a Pagar", f"${max(0, iva_d - iva_c):,}")

    with tab3:
        # Lógica F22 (Consolidada)
        utilidad = bal['Ganancia'].sum() - bal['Pérdida'].sum()
        st.metric("Base Imponible F22", f"${max(0, utilidad):,}")

else:
    st.info("Registra tu primer asiento para habilitar las exportaciones.")
""")

# 3. Lanzamiento
try:
    ngrok.kill()
    public_url = ngrok.connect(8501, proto="http")
    print(f"\\n✅ APP LISTA: {public_url.public_url}\\n")
    subprocess.Popen(["streamlit", "run", "app.py"])
except Exception as e:
    print(f"❌ Error: {e}")

\n✅ APP LISTA: https://jonnie-dilapidated-aylin.ngrok-free.dev\n


In [35]:
# 1. Instalación de herramientas
!pip install streamlit pyngrok python-docx openpyxl -q

import os
import subprocess
from pyngrok import ngrok

# --- CONFIGURACIÓN DE SEGURIDAD ---
TOKEN = "3A5ylB0op0k2iL68de7ztNgBVh5_5Fb2KhvBNQQd5RR3yczLs"
ngrok.set_auth_token(TOKEN)

# 2. Creación del archivo app.py
with open("app.py", "w") as f:
    f.write("""
import streamlit as st
import pandas as pd
from datetime import datetime
from docx import Document
from io import BytesIO

st.set_page_config(page_title="Contabilidad Pro Chile - Cierre Mensual", layout="wide")

# Estilos CSS Profesionales e Impresión
st.markdown('''
<style>
    .t-account { border: 2px solid #333; margin-bottom: 10px; background: white; border-radius: 4px; }
    .t-header { background: #1e3a8a; color: white; text-align: center; font-weight: bold; padding: 5px; }
    .t-body { display: flex; min-height: 50px; }
    .t-side { width: 50%; padding: 5px; font-size: 0.8rem; }
    .t-left { border-right: 2px solid #333; text-align: right; color: #16a34a; }
    .t-right { text-align: left; color: #dc2626; }
    @media print { .no-print, [data-testid="stSidebar"], .stButton, .stDownloadButton { display: none !important; } }
</style>
''', unsafe_allow_html=True)

if 'movimientos' not in st.session_state:
    st.session_state.movimientos = []

# --- FUNCIONES DE EXPORTACIÓN ---
def to_word(df_balance):
    doc = Document()
    doc.add_heading('Reporte de Cierre Mensual - Balance de 8 Columnas', 0)
    table = doc.add_table(rows=1, cols=len(df_balance.columns))
    table.style = 'Table Grid'
    for i, col in enumerate(df_balance.columns):
        table.rows[0].cells[i].text = col
    for _, row in df_balance.iterrows():
        row_cells = table.add_row().cells
        for i, val in enumerate(row):
            row_cells[i].text = str(val)
    buffer = BytesIO()
    doc.save(buffer)
    return buffer.getvalue()

def to_excel(df):
    output = BytesIO()
    with pd.ExcelWriter(output, engine='openpyxl') as writer:
        df.to_excel(writer, index=False, sheet_name='Libro_Diario')
    return output.getvalue()

st.title("🇨🇱 Suite Contable: Gestión, SII y Cierre")

# --- SIDEBAR: HERRAMIENTAS Y CIERRE ---
with st.sidebar:
    st.header("🏁 Operaciones de Período")

    if st.button("🚀 Ejecutar Cierre de Mes", use_container_width=True, type="primary"):
        if st.session_state.movimientos:
            df_temp = pd.DataFrame(st.session_state.movimientos)
            ingresos = df_temp[df_temp['Clase'] == 'Ingreso']['Haber'].sum() - df_temp[df_temp['Clase'] == 'Ingreso']['Debe'].sum()
            gastos = df_temp[df_temp['Clase'] == 'Gasto']['Debe'].sum() - df_temp[df_temp['Clase'] == 'Gasto']['Haber'].sum()
            utilidad = ingresos - gastos
            fecha_cierre = datetime.now().strftime("%Y-%m-%d")

            # Asiento de Cierre
            st.session_state.movimientos.append({
                "Fecha": fecha_cierre, "Cuenta": "CIERRE DE RESULTADOS",
                "Glosa": "Cierre mensual de cuentas nominales", "Debe": ingresos, "Haber": gastos, "Clase": "Pasivo"
            })
            st.session_state.movimientos.append({
                "Fecha": fecha_cierre, "Cuenta": "UTILIDAD DEL EJERCICIO",
                "Glosa": "Resultado del periodo", "Debe": 0, "Haber": utilidad, "Clase": "Pasivo"
            })
            st.success(f"Cierre completado. Utilidad determinada: ${utilidad:,}")
        else:
            st.error("No hay datos para cerrar.")

    st.divider()
    st.header("📥 Descargas")
    if st.session_state.movimientos:
        st.download_button("Excel Completo", to_excel(pd.DataFrame(st.session_state.movimientos)), "contabilidad.xlsx", use_container_width=True)
        if st.button("🖨️ Imprimir Reporte", use_container_width=True):
            st.markdown('<script>window.print();</script>', unsafe_allow_html=True)

    if st.button("🗑️ Resetear Todo", use_container_width=True):
        st.session_state.movimientos = []
        st.rerun()

# --- ENTRADA DE DATOS ---
with st.expander("➕ Registro de Asiento Manual / IVA 19%"):
    col1, col2, col3 = st.columns([1,2,2])
    fecha = col1.date_input("Fecha", datetime.now())
    cuenta = col2.text_input("Cuenta")
    glosa = col3.text_input("Glosa")
    clase = st.selectbox("Clasificación", ["Activo", "Pasivo", "Gasto", "Ingreso"])
    debe = st.number_input("Debe $", min_value=0)
    haber = st.number_input("Haber $", min_value=0)

    if st.button("💾 Guardar + IVA Automático", use_container_width=True):
        if cuenta:
            st.session_state.movimientos.append({"Fecha": str(fecha), "Cuenta": cuenta.upper(), "Glosa": glosa, "Debe": debe, "Haber": haber, "Clase": clase})
            iva = round((debe if debe > 0 else haber) * 0.19)
            es_compra = debe > 0
            st.session_state.movimientos.append({
                "Fecha": str(fecha), "Cuenta": "IVA CRÉDITO" if es_compra else "IVA DÉBITO",
                "Glosa": f"IVA de {glosa}", "Debe": iva if es_compra else 0, "Haber": 0 if es_compra else iva, "Clase": "Activo" if es_compra else "Pasivo"
            })
            st.rerun()

# --- REPORTES ---
if st.session_state.movimientos:
    df = pd.DataFrame(st.session_state.movimientos)
    tab1, tab2, tab3 = st.tabs(["📊 Balance & Libros", "🧾 Formulario 29", "🏛️ Formulario 22"])

    with tab1:
        st.subheader("Balance de 8 Columnas")
        bal = df.groupby(['Cuenta', 'Clase']).agg({'Debe':'sum', 'Haber':'sum'}).reset_index()
        bal['Deudor'] = (bal['Debe'] - bal['Haber']).clip(lower=0)
        bal['Acreedor'] = (bal['Haber'] - bal['Debe']).clip(lower=0)
        bal['Activo'] = bal.apply(lambda x: x['Deudor'] if x['Clase'] == 'Activo' else 0, axis=1)
        bal['Pasivo'] = bal.apply(lambda x: x['Acreedor'] if x['Clase'] == 'Pasivo' else 0, axis=1)
        bal['Pérdida'] = bal.apply(lambda x: x['Deudor'] if x['Clase'] == 'Gasto' else 0, axis=1)
        bal['Ganancia'] = bal.apply(lambda x: x['Acreedor'] if x['Clase'] == 'Ingreso' else 0, axis=1)
        st.table(bal.style.format(precision=0))

        st.download_button("📝 Guardar Balance en Word", to_word(bal), "Balance_Cierre.docx")

    with tab2:
        iva_c = df[df['Cuenta'] == "IVA CRÉDITO"]['Debe'].sum()
        iva_d = df[df['Cuenta'] == "IVA DÉBITO"]['Haber'].sum()
        st.info(f"Impuesto Mensual a Pagar (F29): ${max(0, iva_d - iva_c):,}")

    with tab3:
        utilidad_final = bal['Ganancia'].sum() - bal['Pérdida'].sum()
        st.warning(f"Base Imponible Anual Estimada (F22): ${max(0, utilidad_final):,}")
else:
    st.info("No hay movimientos registrados.")
""")

# 3. Lanzamiento
try:
    ngrok.kill()
    public_url = ngrok.connect(8501, proto="http")
    print(f"\\n✅ ACCESO ONLINE: {public_url.public_url}\\n")
    subprocess.Popen(["streamlit", "run", "app.py"])
except Exception as e:
    print(f"❌ Error: {e}")

\n✅ ACCESO ONLINE: https://jonnie-dilapidated-aylin.ngrok-free.dev\n
